### Purpose 
Exploring a quick way to extract table information from sql files. This use case is suitable for anyone who is dealing with SQL files locally. I will write out a more extensive ReadMe for this.

#### Basic Use Cases
1. Identify tables and variables used within a single SQL script
1. Map out which SQL scripts are using which tables and variables

In [29]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

import re
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
from sql_explorer_v2 import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Locating SQL scripts 

In [30]:
# Ideally, your SQL scripts should all be located in one central folder
# Change this base_path to the path of your own SQL scripts
base_path = os.environ['sql_explorer_folder'].replace("sql_script_explorer/sql_test_scripts/", "script_explorer_scripts/")

# Use this if you have all your SQL scripts exist in one main folder, 
# but scattered into multiple folders within that central folder
# all_sql_scripts = glob(base_path + "**/*.sql", recursive=True)

# Default setting is all the scripts reside in just one folder (Not advisable for production setting)
all_sql_scripts = glob(base_path + "*.sql", recursive=True)
all_sql_scripts = [i.replace(base_path, '') for i in all_sql_scripts]

### Test one script

In [31]:
sql_script_name = all_sql_scripts[4]
with open(base_path + sql_script_name) as filename:
    sql = " ".join(filename.readlines())

script = SQL(sql)
script.clean_whitespaces()
script.lowercase_text()
script.replace_commas_w_newline()
script.split_sql_text_by_newlines()
script.rm_comments()
script.rm_empty_strings()
script.extract_tables()
script.extract_primary_tables()
script.extract_table_variables()
script.clean_up_case_when_vars()
script.clean_date_timestamp_vars()

### Looping through all SQL scripts

In [33]:
main_dict = dict()
for script_name in all_sql_scripts:
    file_name = script_name.split('/')[-1]
    with open(base_path + script_name) as filename:
        sql = " ".join(filename.readlines())

    script = SQL(sql)
    script.clean_whitespaces()
    script.lowercase_text()
    script.replace_commas_w_newline()
    script.split_sql_text_by_newlines()
    script.rm_comments()
    script.rm_empty_strings()
    script.extract_tables()
    script.extract_primary_tables()
    script.extract_table_variables()
    script.clean_up_case_when_vars()
    script.clean_date_timestamp_vars()
    
    main_dict[file_name] = script.raw_sql_tables

### Summary Tables 

In [35]:
summary_table = pd.DataFrame.from_dict(main_dict, orient='index').reset_index()
summary_table.columns = ['script', 'tables']
summary_table['tables'] = [i.split(" | ") for i in summary_table['tables']]
summary_table = summary_table.explode('tables').reset_index(drop=True)
summary_table['tables'] = [i.split('.')[-1] for i in summary_table['tables']]
# summary_table

#### Mapping Script To Tables 

In [20]:
script_to_table = summary_table.groupby('script')['tables'].apply(' | '.join).reset_index()
script_to_table['cnt'] = [len(i.split(" | ")) for i in script_to_table['tables']]
script_to_table.loc[script_to_table['tables'] == '', 'cnt'] = 0
# script_to_table.head()

#### Mapping Tables To Script

In [21]:
table_to_script = summary_table.groupby('tables')['script'].apply(' | '.join).reset_index()
table_to_script['cnt'] = [len(i.split(" | ")) for i in table_to_script['script']]
table_to_script.loc[table_to_script['tables'] == '', 'cnt'] = 0
# table_to_script.head()